In [2]:
import json
import sys
import os

# parent_dir = os.path.abspath(os.getcwd())
# vendor_dir = os.path.join(parent_dir, 'vendor')
# sys.path.append(vendor_dir)

# from tripadvisorCrawler.items import ReviewItem

from items import *
from dataframe import *
    
import matplotlib.pyplot as plt
import numpy as np

with open('data/do.september.11/review_items.jl', 'r') as f:
    raw = f.read()

reviews = as_dataframe(raw)

with open('data/do.september.11/hotel_items.jl','r') as f:
    raw = f.read()

hotels = as_dataframe(raw)

In [3]:
## show case hotels
hotels.head(3)

,batch_id,description,item_id,item_title,site_name,url
0,20160910101527,"Aberdeen Lodge, Dublin: See 447 traveller revi...",https://www.tripadvisor.ie/Hotel_Review-g18660...,\nAberdeen Lodge\n,TripAdvisor,https://www.tripadvisor.ie/Hotel_Review-g18660...
1,20160910101527,"Brooks Hotel, Dublin: See 1,881 traveller revi...",https://www.tripadvisor.ie/Hotel_Review-g18660...,\nBrooks Hotel\n,TripAdvisor,https://www.tripadvisor.ie/Hotel_Review-g18660...
2,20160910101527,"The Marker Hotel, Dublin: See 1,990 traveller ...",https://www.tripadvisor.ie/Hotel_Review-g18660...,\nThe Marker Hotel\n,TripAdvisor,https://www.tripadvisor.ie/Hotel_Review-g18660...


In [4]:
# number of hotels
hotels.shape[0]

157

In [5]:
# show case reviews
reviews.head(3)

,batch_id,item_id,rating,rating_date,rating_percentage,review_id,review_text,review_title,site_name,url,user_id
0,20160910101527,https://www.tripadvisor.ie/Hotel_Review-g18660...,4,2016-08-19T00:00:00,0.75,review_407555899,\nThis is a great location in the city center ...,“Great location”,TripAdvisor,https://www.tripadvisor.ie//ShowUserReviews-g1...,AFCA46113E11EB6873AEF28114293CFF
1,20160910101527,https://www.tripadvisor.ie/Hotel_Review-g18660...,3,2016-08-23T00:00:00,0.50,review_409466696,"\nHotel needs updating, bit shabby, looks bett...",“Would not rate this hotel as 4*”,TripAdvisor,https://www.tripadvisor.ie//ShowUserReviews-g1...,DAA7F593FFF32A4DA781959545562C71
2,20160910101527,https://www.tripadvisor.ie/Hotel_Review-g18660...,5,2016-08-25T00:00:00,1.00,review_410673972,\nMy husband and I were absolutely delighted w...,“the perfect place to stay in Dublin”,TripAdvisor,https://www.tripadvisor.ie//ShowUserReviews-g1...,5F0155A0A1DE2564C8195A07E024DF5B


In [12]:
#summary of rating of dublin hotels
reviews.ix[:,['rating']].describe()

,rating
count,131164.000000
mean,4.103809
std,1.072711
min,1.000000
25%,4.000000
50%,4.000000
75%,5.000000
max,5.000000


In [ ]:
reviewByHotels = reviews.groupby('item_id')
reviewByHotels.agg(['count'])